# Agggregate landcover classes by admin unit

Aggregating landcover by administrative unit isn't simple with desktop tools - each class needs to be aggregated separately and then calculated as a percentage of the total area. Better to do it in python.

In [1]:
import os, sys
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import shape, JOIN_STYLE

import re
import numpy as np
import rasterio
from rasterio.mask import mask
import rasterstats
from rasterstats import zonal_stats

#### Set directories

In [18]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'
vect_in_dir = r'vect_inputs'
vect_out_dir = r'vect_out'
tabular_dir = r'tabular'

Projections

In [3]:
dest_crs = 32642

#### Read in files

KP boundaries

In [4]:
kp = gpd.read_file(r'P:\PAK\GEO\Boundaries\OCHA\pak_admbnda_adm1_ocha_pco_gaul_20181218.shp')

In [5]:
kp = kp[kp['ADM1_EN'] == 'Khyber Pakhtunkhwa']
kp = kp.to_crs(dest_crs)

In [6]:
# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
kp.geometry = kp.buffer(20000)
# kp = kp.to_crs(4326) # change back for clipping of 4326 rasters

In [7]:
kp.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM0_EN,ADM0_PCODE,date,validOn,validTo,geometry
2,27.479368,9.901186,Khyber Pakhtunkhwa,PK2,None,None,None,Pakistan,PK,2018-11-30,2018-12-18,None,"POLYGON ((505910.537 3561734.747, 505840.700 3..."


Admin areas

In [12]:
shp_list = [os.path.join(geo_dir,'Boundaries/KP_Analysis/KP_Analysis_Focus_Tehsils.gpkg')]

In [13]:
shp_list

['P:\\PAK\\GEO\\Boundaries/KP_Analysis/KP_Analysis_Focus_Tehsils.gpkg']

FAO Landcover layers

In [9]:
with rasterio.open(os.path.join(geo_dir,r'Landcover/FAO/kp_lc_v1_AG_cl_32642.tif'),'r') as fao_raw:
    fao_mast, fao_tform = mask(fao_raw, kp.geometry, crop=True, indexes=1)
    fao_mast = fao_mast.astype(np.uint8)
    
    # load and update meta
    fao_prof = fao_raw.profile
    fao_prof['transform'] = fao_tform
    
    # calculate the pixel area in square kilometers
    pix_area = (fao_tform.a * fao_tform.e) / 1000000

In [10]:
# create separate binary arrays for each cropping type
orchard = np.where(fao_mast == 3, 1, 0).astype(np.uint8)
irrig = np.where(fao_mast == 4, 1, 0).astype(np.uint8)
non_irrig = np.where(fao_mast == 5, 1, 0).astype(np.uint8)

In [11]:
pix_area = (fao_tform.a * fao_tform.e) / 1000000

### Aggregate landcover and export

Load in the admin areas and aggregate population + agricultural areas within them. Then export to spatial and tabular formats

In [20]:
for shp in shp_list:
    
    # read in, slim down
    layer = gpd.read_file(shp)
    layer = layer[['ADM3_EN','ADM3_PCODE','ADM2_EN','ADM2_PCODE','ADM1_EN','ADM1_PCODE','geometry']]
    layer = layer.to_crs(dest_crs)
    
    # summarize square kilometers of agricultural land of each type within catchment
    orch_zs = zonal_stats(layer, orchard, affine=fao_tform, nodata=np.nan,stats=['sum'])
    irrig_zs = zonal_stats(layer, irrig, affine=fao_tform, nodata=np.nan,stats=['sum'])
    non_irrig_zs = zonal_stats(layer, non_irrig, affine=fao_tform, nodata=np.nan,stats=['sum'])
    
    # add pixel sum column to layer and multiply it by pixel area factored in terms of square kilometers
    
    layer['area_sqkm'] = (layer.geometry.area / 1000000) 
    
    layer['orch_sqkm'] = np.abs(pd.DataFrame(orch_zs)['sum'] * pix_area)
    layer['irrig_sqkm'] = np.abs(pd.DataFrame(irrig_zs)['sum'] * pix_area)
    layer['non_irrig_sqkm'] = np.abs(pd.DataFrame(non_irrig_zs)['sum'] * pix_area)

    layer['orch_pct'] = (layer['orch_sqkm'] / layer['area_sqkm']) * 100
    layer['irrig_pct'] = (layer['irrig_sqkm'] / layer['area_sqkm']) * 100
    layer['non_irrig_pct'] = (layer['non_irrig_sqkm'] / layer['area_sqkm']) * 100
    
    # export
    layer.to_file(os.path.join(data_dir,vect_out_dir,f'KP_Admins_Agricultural.gpkg'),layer=f'Tehsils_Agricultural_{dest_crs}',driver="GPKG")
    layer.to_crs(4326).to_file(os.path.join(data_dir,vect_out_dir,f'KP_Admins_Agricultural.gpkg'),layer=f'Tehsils_Agricultural_4326',driver="GPKG")
    layer.drop('geometry',axis=1).to_csv(os.path.join(data_dir,tabular,f'KP_Tehsil_Agricultural_Areas.csv'))